In [1]:
# !pip install rank_bm25

In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
%cd ../

/workspace


In [3]:
# 라이브러리 임포트
import json

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from rank_bm25 import BM25Okapi
from tqdm.auto import tqdm

In [4]:
# 데이터 로드
train = json.load(open("./data/train.json", "r", encoding="utf8"))
test_data = json.load(open("./data/test_data.json", "r", encoding="utf8"))
test_question = pd.read_csv("./data/test_questions.csv", encoding="utf8")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf8")

# bm25 Train Set 성능 확인

### DATA

In [12]:
train['data'][0]['paragraphs'][0]

{'paragraph_id': 'PARS_eWP8RZc99q',
 'context': '방사성핵종 별 공기 부피당 방사능 농도(Bq/m3 )는 부피당 분진 농도(μg/L)와 질량당 방사능 농도(Bq/kg) 값을 이용하여 계산 가능하다. 분진이 흩날릴 가능성이 있는 작업 지점에 설치한 광학입자계수기(OPC, Optical Particle Counter)를 활용한 계측과 채취시료의 방사능분석 의뢰 등을 통하여 이 값들을 각각 얻을 수 있다. 선량환산인자(DCF, Dose Conversion Factor)는 원료물질 등이 함유한 방사성핵종의 종류 외에 분진 입자의 크기 및 분포, 밀도, 기하학적 모양, 흡수형태 등 물리화학적 형태에 따라 그 값이 크게 달라진다. 입자의 크기와 분포는 앞서 언급한 OPC를 분석하여 얻은 공기역학적 방사능 중간직경(AMAD, Activity Median Aerodynamic Diameter)과 직경분포에 따른 분산(GSD, Geometric Standard Deviation)으로 결정되며, 입자의 밀도는 Pycnometer 등의 밀도측정장비로 얻을 수 있다. 모양인자는 1로 통일하여 적용되는데 이 값은 1에 가까운 구형일수록 보수적인 평가가 이루어진다는 해외 연구결과를 바탕으로 볼 때 합리적이다.',
 'qas': [{'question_id': 'QUES_VzfDEdaQOz',
   'question': '공기역학적 방사능 중간직경은 무엇을 분석하여 얻었어'},
  {'question_id': 'QUES_RtPCd1sgHS',
   'question': '무엇을 검토하여 공기역학적 방사능 중간직경을 얻었니'},
  {'question_id': 'QUES_hRDQkUSHzf',
   'question': '무엇을 분석하여 구한 것이 공기역학적 방사능 중간직경이지'}]}

In [21]:
number_of_contents = len(train["data"])
bound = number_of_contents

contexts = []
contexts2 = []
questions = []

for i in range(bound):
    for j in range(len(train["data"][i]["paragraphs"])):
        if len(train["data"][i]["paragraphs"][j]["qas"]) > 0:
            contexts.append(train["data"][i]["paragraphs"][j]["context"])
            contexts2.append(
                {
                    "id": train["data"][i]["paragraphs"][j]["paragraph_id"],
                    "contents": train["data"][i]["paragraphs"][j]["context"],
                }
            )
            questions.append(train["data"][i]["paragraphs"][j]["qas"][0]["question"])

In [22]:
with open("./data/index/train2.json", "w", encoding="utf8") as f:
    json.dump(contexts2, f)

In [5]:
contexts[:3]

['방사성핵종 별 공기 부피당 방사능 농도(Bq/m3 )는 부피당 분진 농도(μg/L)와 질량당 방사능 농도(Bq/kg) 값을 이용하여 계산 가능하다. 분진이 흩날릴 가능성이 있는 작업 지점에 설치한 광학입자계수기(OPC, Optical Particle Counter)를 활용한 계측과 채취시료의 방사능분석 의뢰 등을 통하여 이 값들을 각각 얻을 수 있다. 선량환산인자(DCF, Dose Conversion Factor)는 원료물질 등이 함유한 방사성핵종의 종류 외에 분진 입자의 크기 및 분포, 밀도, 기하학적 모양, 흡수형태 등 물리화학적 형태에 따라 그 값이 크게 달라진다. 입자의 크기와 분포는 앞서 언급한 OPC를 분석하여 얻은 공기역학적 방사능 중간직경(AMAD, Activity Median Aerodynamic Diameter)과 직경분포에 따른 분산(GSD, Geometric Standard Deviation)으로 결정되며, 입자의 밀도는 Pycnometer 등의 밀도측정장비로 얻을 수 있다. 모양인자는 1로 통일하여 적용되는데 이 값은 1에 가까운 구형일수록 보수적인 평가가 이루어진다는 해외 연구결과를 바탕으로 볼 때 합리적이다.',
 '2015년 장비 취득 중 α·β counter는 표면오염도 검사를 위한 측정장비로써 문지름검사 시 험지를 측정시료로 사용할 수 있어 면적에 따른 방사선의 확인이 가능하여 가공제품 안전기준 중 공기누출 및 신체전이여부의 정확한 평가를 위해 취득하였다. 전처리 장비는 가공제품 시료 수 증가 및 가공제품의 특성상 회화율이 높아 대용량의 전처리가 필요하다고 판단되어 기존의 장비를 증량하였으며 실험초자의 세척 장비는 방사능분석 실험의 특 성 상 초자 또는 장비의 오염은 분석 결과의 치명적인 오류를 초래할 수 있어 이에 대한 영향을 최소화하기 위하여 해당 장비들을 취득하였다. 또한 방사능분석의 방사성 추적자 용액( 229Th, 232U)과 알파핵종의 분석과정에서 사용되는 HNO3, HCl, HF 와 같은 강 산성 용액들의 위험으

In [6]:
questions[:3]

['공기역학적 방사능 중간직경은 무엇을 분석하여 얻었어',
 '신체 전이 여부와 공기 누출의 확실한 평가를 위해 있어야 할 측정장비가 뭐야',
 '생활제품부터 건축자재에 이르기까지 다양한 가공제품은 무엇이 함유된 원료를 사용하여 만드니']

### Model

In [7]:
bm25 = BM25Okapi([doc.split(' ') for doc in contexts])

In [11]:
len(questions)

123552

In [9]:
def get_top_n(q, n=10):
    return bm25.get_top_n(q, contexts, n=n)

In [ ]:
answer = Parallel(n_jobs=8)(
    delayed(get_top_n)(q) for q in tqdm([question.split(" ") for question in questions])
)

In [ ]:
answer = []
for q in tqdm([question.split(' ') for question in questions]):
    answer.append(bm25.get_top_n(q, contexts, n=10))

In [ ]:
accumulated_RR = 0

for index, value in enumerate(answer):
    for index_, value_ in enumerate(answer[index]):
        if value_ == contexts[index]:
            accumulated_RR += 1/(1 + index_)

MRR = accumulated_RR/(len(answer))
MRR

# bm25 Test Set 추론

### DATA

In [ ]:
questions = list(test_question['question_text'])

contexts = []
par_ids = []

for i in range(len(test_data['data'])):
    for j in range(len(test_data['data'][i]['paragraphs'])):
        contexts.append(test_data['data'][i]['paragraphs'][j]['context'])
        par_ids.append(test_data['data'][i]['paragraphs'][j]['paragraph_id'])

In [ ]:
questions[:3]

In [ ]:
contexts[:3]

In [ ]:
par_ids[:3]

### Model

In [ ]:
bm25 = BM25Okapi([doc.split(' ') for doc in contexts])

In [ ]:
answer = []
for q in [question.split(' ') for question in questions]:
    answer.append(bm25.get_top_n(q, contexts, n=10))

In [ ]:
for i, a in enumerate(answer):
    for j, b in enumerate(a):
        for k, c in enumerate(contexts):
            if b == c:
                if j == 0:
                    prediction = par_ids[k]
                elif j < 9:
                    prediction = prediction + ',' + par_ids[k]
                else:
                    prediction = prediction + ',' + par_ids[k]
                    sample.loc[i, 'paragraph_id'] = prediction
sample

In [ ]:
sample.to_csv('##### prediction 파일 경로 #####', index=False)